# Global Setup

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sys
from pathlib import Path

import json
import pandas as pd
import random


import tqdm

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import huggingface_hub

import pickle

import pytorch_lightning as pl

import data
import models

In [5]:
DATA_FOLDER = Path("starter")

In [21]:
from argparse import Namespace

args = Namespace(
    batch_size=32,
    epochs=10,
    max_len=128,
    max_len_vader=40,
    use_bert=False,
    use_cnn=False,
    use_vader=False,
)

In [18]:
try:
    yelp_dataset = pickle.load("datamodule.pkl")
except:
    yelp_dataset = data.YelpDataModule(
        args, data_path="starter/yelp_review_training_dataset.jsonl"
    )
    with open("datamodule.pkl", "wb") as f:
        pickle.dump(yelp_dataset, f)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alokbeniwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


creating dataset


Using cache found in /Users/alokbeniwal/.cache/torch/hub/huggingface_pytorch-transformers_master
533581it [13:07, 677.79it/s]


In [22]:
model = models.LanguageModel(
    args,
)

Using cache found in /Users/alokbeniwal/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /Users/alokbeniwal/.cache/torch/hub/huggingface_pytorch-transformers_master


Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

1 100


In [ ]:
trainer = pl.Trainer()
trainer.fit(model, datamodule=yelp_dataset)
trainer.validate(model, datamodule=yelp_dataset)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alokbeniwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
